> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [1]:
### Escreva sua resposta aqui
import pandas as pd

# URL direta para download do CSV raw no GitHub
url = 'https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/salaries.csv'

# Carregar os dados
df = pd.read_csv(url)

# Mostrar as 5 primeiras linhas para conhecer o dataset
print(df.head())

# Mostrar informações gerais (colunas, tipos e não nulos)
print(df.info())

  FIRST NAME   LAST NAME SEX         DOJ CURRENT DATE DESIGNATION   AGE  \
0     TOMASA       ARMEN   F   5-18-2014   01-07-2016     Analyst  21.0   
1      ANNIE         NaN   F         NaN   01-07-2016   Associate   NaN   
2      OLIVE        ANCY   F   7-28-2014   01-07-2016     Analyst  21.0   
3     CHERRY     AQUILAR   F  04-03-2013   01-07-2016     Analyst  22.0   
4       LEON  ABOULAHOUD   M  11-20-2014   01-07-2016     Analyst   NaN   

   SALARY        UNIT  LEAVES USED  LEAVES REMAINING  RATINGS  PAST EXP  
0   44570     Finance         24.0               6.0      2.0         0  
1   89207         Web          NaN              13.0      NaN         7  
2   40955     Finance         23.0               7.0      3.0         0  
3   45550          IT         22.0               8.0      3.0         0  
4   43161  Operations         27.0               3.0      NaN         3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2639 entries, 0 to 2638
Data columns (total 13 columns)

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [7]:
### Escreva sua resposta aqui
import enum
from sqlalchemy import Column, Integer, String, Float, Date, Enum
from sqlalchemy.orm import declarative_base

Base = declarative_base()

# Enum para Sexo
class SexEnum(enum.Enum):
    F = "F"
    M = "M"

# Enum para Designation (cargo)
class DesignationEnum(enum.Enum):
    Analista = "Analista"
    Analista_Senior = "Analista Sênior"
    Gerente = "Gerente"

# Enum para Unidade (UNIT)
class UnitEnum(enum.Enum):
    TI = "TI"
    Financas = "Finanças"
    Marketing = "Marketing"

class ProfissionalDados(Base):
    __tablename__ = 'profissionais_dados'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(Enum(SexEnum))
    doj = Column(Date)  # Data de ingresso
    current_date = Column(Date)  # Data atual/referência
    designation = Column(Enum(DesignationEnum))
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(Enum(UnitEnum))
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)

    def __repr__(self):
        return f"<ProfissionalDados(id={self.id}, nome={self.first_name} {self.last_name}, cargo={self.designation.value})>"

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [8]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine

# Criar engine para banco SQLite chamado salarios.db (arquivo local)
engine = create_engine('sqlite:///salarios.db')

# Teste: tentar conectar (vai criar o arquivo se não existir)
connection = engine.connect()
print("Conexão com o banco SQLite 'salarios.db' criada com sucesso.")

# Fechar conexão
connection.close()

Conexão com o banco SQLite 'salarios.db' criada com sucesso.


#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [9]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Supondo que você já tenha definido o Base e a classe Salario no Q2

# Criar engine para o banco salarios.db (se já não tiver criado)
engine = create_engine('sqlite:///salarios.db')

# Criar as tabelas no banco (baseado nas classes ORM)
Base.metadata.create_all(engine)

print("Tabelas criadas com sucesso no banco salarios.db")

Tabelas criadas com sucesso no banco salarios.db


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [10]:
### Escreva sua resposta aqui
import pandas as pd
from sqlalchemy import create_engine

# 1. Carregue o CSV novamente (caso não tenha carregado ainda)
df = pd.read_csv('salaries.csv')

# 2. Criar a engine do banco SQLite (caso não tenha criado ainda)
engine = create_engine('sqlite:///salarios.db')

# 3. Inserir os dados do DataFrame na tabela do banco
# Nome da tabela deve ser o mesmo que você criou no ORM, por exemplo: 'salary' (confirme o nome)
df.to_sql('salary', con=engine, if_exists='append', index=False)

print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!


#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [13]:
### Execute aqui sua query SQL com SQLAlchemy
# Criar engine
engine = create_engine('sqlite:///salarios.db')

# Criar tabela
Base.metadata.create_all(engine)

# Inserir dados
df.to_sql('salaries', con=engine, if_exists='append', index=False)

# Consultar
from sqlalchemy import text
with engine.connect() as conn:
    query = text("""
        SELECT DESIGNATION,
               MIN(SALARY / 12.0) AS min_salary_monthly,
               MAX(SALARY / 12.0) AS max_salary_monthly,
               AVG(SALARY / 12.0) AS avg_salary_monthly
        FROM salaries
        GROUP BY DESIGNATION
        ORDER BY DESIGNATION
    """)
    result = conn.execute(query)
    for row in result:
        print(row)

('Analyst', 3333.4166666666665, 4165.0, 3751.675987685993)
('Associate', 5846.166666666667, 8300.25, 7266.915094339623)
('Director', 17832.25, 32342.666666666668, 23914.265625)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134832)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129032)


In [14]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas
import pandas as pd
from sqlalchemy import create_engine

# Supondo que o engine já esteja criado
# engine = create_engine('sqlite:///salarios.db')

query = """
    SELECT DESIGNATION,
           MIN(SALARY / 12.0) AS min_salary_monthly,
           MAX(SALARY / 12.0) AS max_salary_monthly,
           AVG(SALARY / 12.0) AS avg_salary_monthly
    FROM salaries
    GROUP BY DESIGNATION
    ORDER BY DESIGNATION
"""

# Executa a query com pandas e SQLAlchemy
df = pd.read_sql_query(query, engine)

print(df)

      DESIGNATION  min_salary_monthly  max_salary_monthly  avg_salary_monthly
0         Analyst         3333.416667         4165.000000         3751.675988
1       Associate         5846.166667         8300.250000         7266.915094
2        Director        17832.250000        32342.666667        23914.265625
3         Manager         8343.666667        12407.500000        10522.716049
4  Senior Analyst         4170.333333         5830.500000         4991.778792
5  Senior Manager        12614.416667        16631.416667        14888.689516


In [17]:
### Execute aqui sua query com SQLAlchemy ORM
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date, Enum
import enum

Base = declarative_base()

class SexEnum(enum.Enum):
    F = 'F'
    M = 'M'

class DesignationEnum(enum.Enum):
    Analyst = 'Analyst'
    Associate = 'Associate'
    Director = 'Director'
    Manager = 'Manager'
    Senior_Analyst = 'Senior Analyst'
    Senior_Manager = 'Senior Manager'

class UnitEnum(enum.Enum):
    TI = 'TI'
    Finance = 'Finance'
    Marketing = 'Marketing'

class Salary(Base):
    __tablename__ = 'salaries'
    ID = Column(Integer, primary_key=True, autoincrement=True)
    FIRST_NAME = Column(String)
    LAST_NAME = Column(String)
    SEX = Column(Enum(SexEnum))
    DOJ = Column(Date)
    CURRENT_DATE = Column(Date)
    DESIGNATION = Column(Enum(DesignationEnum))
    AGE = Column(Integer)
    SALARY = Column(Float)
    UNIT = Column(Enum(UnitEnum))
    LEAVES_USED = Column(Integer)
    LEAVES_REMAINING = Column(Integer)
    RATINGS = Column(Float)
    PAST_EXP = Column(Float)

C:\Users\PDBD217\AppData\Local\Temp\ipykernel_24280\1324546842.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
